# Likelihood Optimization of gas Kinematics in IFUs (LOKI)
## Fitting example: MIRI + PSF model

Michael Reefe

This example notebook provides a tutorial on how to run LOKI on a multi-channel MIRI/MRS cube, including a PSF model to separate out light from a bright quasar.

In this example, we'll utilize the multi-processing capabilities of the code.

In [ ]:
using Distributed
procs = addprocs(Sys.CPU_THREADS, exeflags="--heap-size-hint=4G")

@everywhere begin
    using Pkg
    Pkg.activate(dirname(@__DIR__))
    Pkg.instantiate()
    Pkg.precompile()
    using Loki
    using Unitful 
end

Now we want to load in our data. For this example, we'll be using the channel 1-4 data for NGC 7469, which is located in the same folder as this notebook. The JWST reduced data does not include a redshift, so we must provide the redshift ourselves.  We can use the `from_fits` function to load in the JWST-formatted FITS files, along with the redshift.

In [ ]:
# The redshift of the target object: NGC 7469
z = 0.016317
# The semicolon at the end suppresses printing the output Observation object, which is long and not very enlightening
obs = from_fits(["Level3_ch1-long_s3d.fits.gz", 
                 "Level3_ch1-medium_s3d.fits.gz", 
                 "Level3_ch1-short_s3d.fits.gz",
                 "Level3_ch2-long_s3d.fits.gz",
                 "Level3_ch2-medium_s3d.fits.gz",
                 "Level3_ch2-short_s3d.fits.gz",
                 "Level3_ch3-long_s3d.fits.gz",
                 "Level3_ch3-medium_s3d.fits.gz",
                 "Level3_ch3-short_s3d.fits.gz",
                 "Level3_ch4-long_s3d.fits.gz",
                 "Level3_ch4-medium_s3d.fits.gz",
                 "Level3_ch4-short_s3d.fits.gz"], z);

Next, we create some variables that we will use later. We will be fitting multi-channel data, and we can take the `name` property from the Observation object we just loaded in to get the name of the target. Here, `run_name` is just a unique identifier that we will use for this run.

In [ ]:
channel = 0   # since we are including data from all 4 channels, this is just a placeholder
nm = replace(obs.name, " " => "_") 
run_name = "$(nm)_ch$(channel)_psf_model"

Before fitting, we want to do some pre-processing on the data. We want to convert the data to the rest-frame, mast out / interpolate any bad pixels, and replace the JWST pipeline-generated errors with some more realistic ones.  We also need to create our PSF model that we will use to model the bright nuclear point-source from the AGN.  All of this is achieved in the next block of code. This is also where we will combine data from multiple channels into a single cube using the `combine_channels!` function.

In [ ]:
if isfile("$nm.channel$channel.rest_frame.fits")
    # If we've already performed this step in a previous run, just load in the pre-processed data
    obs = from_fits(["$nm.channel$channel.rest_frame.fits"], obs.z);
    
else
    # First, generate the PSF models
    generate_psf_model!(obs)

    # Convert to rest-frame wavelength vector, and mask out bad spaxels
    correct!(obs)

    # Fit cubic spline to the PSF
    for band in (:A, :B, :C)
        for channel in (1, 2, 3, 4)
            chband = Symbol(band, channel)
            splinefit_psf_model!(obs.channels[chband], 100)
        end
    end

    # Reproject the sub-channels onto the same WCS grid and combine them into one full channel
    # - The [:A1, :B1, :C1, ...] vector gives the names of each channel to concatenate. By default, JWST subchannels are
    #   given labels of "A" for short, "B" for medium, and "C" for long, followed by the channel number. Make sure you 
    #   list them in order of increasing wavelength, otherwise the resampling won't work correctly. 
    # - The "out_id" argument will determine the label given to the combined channel data. 
    combine_channels!(obs, [:A1,:B1,:C1,:A2,:B2,:C2,:A3,:B3,:C3,:A4,:B4,:C4], out_id=channel, order=1, 
        adjust_wcs_headerinfo=true, extract_from_ap=0., max_λ=17.2u"μm")

    # rotate to the RA/Dec axes on the sky
    rotate_to_sky_axes!(obs.channels[channel])

    # We interpolate any rogue NaNs using a linear interpolation, since the MPFIT minimizer does not handle NaNs well.
    interpolate_nans!(obs.channels[channel])

    # Finally, we calculate the statistical errors (i.e. the standard deviation of the residuals with a cubic spline fit)
    # and replace the errors in the cube with these, since the provided errors are typically underestimated.
    # You can skip this step if you wish to use the default errors.
    calculate_statistical_errors!(obs.channels[channel])
    
    # Save the pre-processed data as a FITS file so it can be quickly reloaded later
    save_fits(".", obs, [channel]);
end

We now need to create the full 3D nuclear template out of the PSF model.  Conceptually, all this does is multiply the normalized PSF with the spectrum of the brightest spaxel.

In [ ]:
nuc_temp = generate_nuclear_template(obs.channels[channel], 0.)

Finally, we create the `CubeFitter` object and call the `fit_cube!` function to fit the data. We've set a few more additional options in the CubeFitter here.  In particular, pay attention to the "templates" and "template_names" arguments, which specify the 3D nuclear template model that we created above, and give it a unique identifier.  For more information about what the other arguments do, refer to the "Usage" section in the README file.

Note: we're doing the whole shebang for this example, so it may take a few hours to complete!

In [ ]:
# To see a full list of keyword arguments, please refer to the docstring, which can be accessed by typing `?CubeFitter` in the command
# line after importing Loki.
cube_fitter = CubeFitter(
    obs.channels[channel], 
    obs.z, 
    run_name; 
    parallel=true, 
    parallel_strategy="pmap",
    plot_spaxels=:pyplot, 
    plot_maps=true, 
    save_fits=true,
    silicate_absorption="d+",
    extinction_screen=true, 
    use_pah_templates=true, 
    fit_sil_emission=false, 
    fit_stellar_continuum=false, 
    save_full_model=true, 
    map_snr_thresh=3., 
    templates=nuc_temp, 
    template_names=["nuclear"], 
    subtract_cubic_spline=true,
)

# Call the fit_cube! function on the cube_fitter object
fit_cube!(cube_fitter)

In [ ]:
# Stop the worker processes
rmprocs(procs)

And the results can be found in the "output_[run_name]" directory, just like the other examples!  Here is a showcase of a few of the fits to some individual spaxels across various locations:

![](./NGC7469.spaxel_10_31.png)
![](./NGC7469.spaxel_25_26.png)
![](./NGC7469.spaxel_27_26.png)
![](./NGC7469.spaxel_32_12.png)

The orange line shows the final model.  The decomposed components of the model consist of:
- Thermal dust continuum, in gray
- The AGN PSF model, in dark green
- PAHs, in blue
- Emission lines, in purple
- Extinction, in dotted gray (read from the right axis)

If you're interested in what the 2D parameter maps should looks like from this fit, check out the README file.  The examples shown there are from this model.

Notice that in some of the fits, the model does not do too well at reproducing the continuum on the far left side (the shortest wavelengths).  This is because we neglected to include any continuum component that could be important here, such as either a hot dust component or a stellar continuum.  One way to alleviate this would be by changing the `fit_stellar_continuum` option in the `CubeFitter` to true.  If you do so, make sure you have installed the python FSPS library!  Or you could instead set `fit_sil_emission` to true to fit a hot silicate dust emission component.  This will also likely change the recovered amplitudes on the AGN PSF template since they can become degenerate at these short wavelengths, so just be aware of that.  A good sanity check for this is to look at the fit results for the brightest spaxel (which should be close to the center of the cube, somewhere around x,y=25,25) - the PSF model is based on this spectrum, so the model should be pretty much 100% composed of the dark green AGN PSF template in this spaxel.  If it's not, you might need to look into either constraining the PSF template amplitude parameters or removing the degenerate model components.

Now that you've run this model, be sure to check out the continuation in "example_MIRI_qso_model", which will take the results of this model and use them to reconstruct a 1D quasar spectrum.